In [1]:
import os
import os.path as path

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

In [2]:
MODEL_NAME = 'xo_differ'

BATCH_SIZE = 16

In [3]:
def build_model(x, keep_prob, y_, output_node_name):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    # 28*28*1

    conv1 = tf.layers.conv2d(x_image, 64, 3, 1, 'same', activation=tf.nn.relu)
    # 28*28*64
    pool1 = tf.layers.max_pooling2d(conv1, 2, 2, 'same')
    # 14*14*64

    conv2 = tf.layers.conv2d(pool1, 128, 3, 1, 'same', activation=tf.nn.relu)
    # 14*14*128
    pool2 = tf.layers.max_pooling2d(conv2, 2, 2, 'same')
    # 7*7*128

    conv3 = tf.layers.conv2d(pool2, 256, 3, 1, 'same', activation=tf.nn.relu)
    # 7*7*256
    pool3 = tf.layers.max_pooling2d(conv3, 2, 2, 'same')
    # 4*4*256

    flatten = tf.reshape(pool3, [-1, 4*4*256])
    fc = tf.layers.dense(flatten, 1024, activation=tf.nn.relu)
    dropout = tf.nn.dropout(fc, keep_prob)
    logits = tf.layers.dense(dropout, 2)
    outputs = tf.nn.softmax(logits, name=output_node_name)

    # loss
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=logits))

    # train step
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    # accuracy
    correct_prediction = tf.equal(tf.argmax(outputs, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged_summary_op = tf.summary.merge_all()

    return train_step, loss, accuracy, merged_summary_op

In [4]:
input_node_name = 'input'
keep_prob_node_name = 'keep_prob'
output_node_name = 'output'

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 28*28], name=input_node_name)
keep_prob = tf.placeholder(tf.float32, name=keep_prob_node_name)
y_ = tf.placeholder(tf.float32, shape=[None, 2])

In [6]:
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [7]:
def get_data():
    n1 = np.load("O.npy")
    n2 = np.load("X.npy")
    print(n2.shape)
    print(n1.shape)
    x = n1 
    x = np.concatenate((x,n2),axis=0)
    length1 = n1.shape[0]
    print(length1)
    y = np.zeros((15939), dtype=int)
    length2 = n2.shape[0]
    print(length2)
    y[0:length1] = 0
    y[length1:length1+length2] = 1
    
    print(y)


    # one hot encode
    encoded = to_categorical(y)
    print(encoded.shape)
    return x,encoded

In [8]:
batch_x, batch_y = get_data()

print(batch_x.shape)
print(batch_y.shape)

i = np.arange(15939)
s = np.arange(i.shape[0])
np.random.shuffle(s)

batch_x = batch_x[s]
batch_y = batch_y[s]

(6963, 784)
(8976, 784)
8976
6963
[0 0 0 ... 1 1 1]
(15939, 2)
(15939, 784)
(15939, 2)


In [9]:
batch_y[8:34]

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [10]:
train_step, loss, accuracy, merged_summary_op = build_model(x, keep_prob,y_, output_node_name)

In [11]:
NUM_STEPS = 10

In [12]:
from tqdm import tqdm
init_op = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)

    tf.train.write_graph(sess.graph_def, 'out',
        MODEL_NAME + '.pbtxt', True)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter('logs/',
        graph=tf.get_default_graph())

    for _ in range(3):
        for i in tqdm(range(0,batch_x.shape[0],50)):
            x_data = batch_x[i:i+50]
            y_data = batch_y[i:i+50]

            _, c = sess.run([train_step, loss],
                feed_dict={x: x_data, y_: y_data, keep_prob: 0.5})
            
        
        
        train_loss = loss.eval(feed_dict={
            x: x_data, y_: y_data, keep_prob: 1.0})
        print("Loss : "+ str(train_loss))
        
        train_accuracy = accuracy.eval(feed_dict={
            x: x_data, y_: y_data, keep_prob: 1.0})
        print("Accuracy: "+ str(train_accuracy))

    saver.save(sess, 'out/' + MODEL_NAME + '.chkp')

print("training finished!")

100%|████████████████████████████████████████| 319/319 [01:58<00:00,  2.70it/s]


Loss : 0.010482296
Accuracy: 1.0


100%|████████████████████████████████████████| 319/319 [02:04<00:00,  2.56it/s]


Loss : 6.215855e-05
Accuracy: 1.0


100%|████████████████████████████████████████| 319/319 [02:03<00:00,  2.59it/s]


Loss : 7.244242e-05
Accuracy: 1.0
training finished!


In [ ]:
freeze_graph.freeze_graph('out/' + MODEL_NAME + '.pbtxt', None, False,
    'out/' + MODEL_NAME + '.chkp', output_node_name, "save/restore_all",
    "save/Const:0", 'out/frozen_' + MODEL_NAME + '.pb', True, "")

input_graph_def = tf.GraphDef()
with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
    input_graph_def.ParseFromString(f.read())

input_node_names = ['input']
    
output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        input_graph_def, input_node_names, [output_node_name],
        tf.float32.as_datatype_enum)

with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
    f.write(output_graph_def.SerializeToString())